## Tensorflow tutorial. ##

In [2]:
import tensorflow as tf

Let's work on building a computational graph. We will then run the computational graph.

In [4]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # implicit tf.float32
print node1, node2

Tensor("Const_3:0", shape=(), dtype=float32) Tensor("Const_4:0", shape=(), dtype=float32)


When we print the nodes, notice it didn't return any values. We need to start a **session** before we can actually evaluate the nodes.

In [5]:
sess = tf.Session()
print sess.run([node1, node2])

[3.0, 4.0]


We can also use more complicated operations, which also turn into nodes in our graph.

In [6]:
node3 = tf.add(node1, node2)
print "node3: ", node3
print "sess.run(node3):", sess.run(node3)

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


Here, we used constant values, but we can make it so that our graph accepts external inputs, which are known as **placeholders**.

In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # We can use + instead of tf.add

Let's run our model and give it input parameters by specifying a **feed_dict** of parameters.

In [8]:
print sess.run(adder_node, {a: 3, b:4.5})

7.5


In [9]:
print sess.run(adder_node, {a: [1,3], b:[2,4]})

[ 3.  7.]


In [11]:
add_and_triple = adder_node * 3.

A more complex operation.

In [12]:
print sess.run(add_and_triple, {a:3, b:4.5})

22.5


We may also have to be able to modify certain parts of the graph in order to train a model. We will use **variables** to do this. They have a type and initial value.

In [14]:
W = tf.Variable([.3], tf.float32) # weight
b = tf.Variable([-.3], tf.float32) # bias
x = tf.placeholder(tf.float32) # placeholder for input
linear_model = W * x + b

Variables are not initialized when you call tf.Variable. Instead, we will have to use an explicit operation to initialize all variables in the graph.

In [15]:
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
print sess.run(linear_model, {x:[1,2,3,4]})

[ 0.          0.30000001  0.60000002  0.90000004]


Now we've evaluated our model, but traditionally it is helpful to know the true values and evaluate our model based on that using some form of a loss function.

In [18]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # sum over all examples
print sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})

23.66


We can assign values to variables using the tf.assign function. Let's change W and b to produce a loss of zero.

In [19]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})

0.0


Now we can turn to machine learning, using optimizers along with the gradient descent method to learn new weights and biases to improve our model. We will use the tf.train API.

In [20]:
optimizer = tf.train.GradientDescentOptimizer(0.01) # 0.01 is the learning rate
train = optimizer.minimize(loss)

In [21]:
sess.run(init)
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


We can run the full model here.

In [ ]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

## Keras tutorial. ##

Let's start with the Keras sequential model, where we add layers on top of each other.

In [4]:
from keras.models import Sequential

Using TensorFlow backend.


In [5]:
model = Sequential()

In [6]:
from keras.layers import Dense, Activation

In [9]:
model.add(Dense(units=64, input_dim=100)) # fully-connected
model.add(Activation('relu')) # ReLu activation
model.add(Dense(units=10))
model.add(Activation('softmax')) # softmax at end

We can also combine this into a single step.

In [14]:
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax')
])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 25,450.0
Trainable params: 25,450.0
Non-trainable params: 0.0
_________________________________________________________________


Before running our model, we'll have to compile our model to control the learning process. Keras allows for fine control of the parameters for training using the **compile** function.

In [10]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

Other optimizers include 'rmsprop' and 'adagrad'.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

We then iterate over the training data in batches using the **fit** command.

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32)
# or we can use another way...
model.train_on_batch(x_batch, y_batch)

We can also evaluate our performance using **evaluate**.

In [ ]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)

We can generate predictions on new data using **predict**.

In [ ]:
classes = model.predict(x_test, batch_size=128)

Let's look at some examples.

In [17]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

import numpy as np
data = np.random.random((1000, 100)) # 100 training examples of dimension 100
labels = np.random.randint(2, size=(1000,1))

# Train the model iterating over batches of 32
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.7103 - acc: 0.5150      
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7004 - acc: 0.5060     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.6933 - acc: 0.5350     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.6905 - acc: 0.5440     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6847 - acc: 0.5610     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6803 - acc: 0.5600     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6775 - acc: 0.5710     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6724 - acc: 0.5910     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6680 - acc: 0.5910     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6656 - acc: 0.5940     


In [23]:
from keras.utils import to_categorical # convert input label to one-hot encoded vector